In [1]:
import os
import cv2
import numpy as np

In [2]:
def feature_object_detection(org_img, gray_img, frame, gray_frame, min_match_number) :
    img_kpts, img_desc = cv2.SIFT_create().detectAndCompute(gray_img, None)
    frame_kpts, frame_desc = cv2.SIFT_create().detectAndCompute(gray_frame, None)
    matches = cv2.BFMatcher().knnMatch(img_desc, frame_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.68*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])

    if len(good_matches) > min_match_number :
        src_pts = np.float32([ img_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ frame_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        Homography, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 0.0)
        
        height, width = org_img.shape[:2]
        img_points = np.float32([[0, 0], [0, height-1], [width-1, height-1], [width-1, 0]]).reshape(-1,1,2)
        book_points = cv2.perspectiveTransform(img_points, Homography)

        detected_book = cv2.polylines(frame, [np.int32(book_points)], True, frame_color, 2, cv2.LINE_AA)

        return detected_book
    else :
        return frame
    

In [3]:
frame_color = (188,132,255)

In [4]:
org_img = cv2.imread('./images/final_exam/Templates/Template-3.png')
blur_img = cv2.bilateralFilter(org_img,9,75,75)
blur_img = cv2.GaussianBlur(blur_img,(5,5),cv2.BORDER_DEFAULT)
gray_img = cv2.cvtColor(blur_img, cv2.COLOR_BGR2GRAY)

org_vid = cv2.VideoCapture('./videos/final_exam/Dataset-2/left_output.avi')
kernel = np.array([[0, 0, 0],
                   [0, 2.5, 0],
                   [0, 0, 0]])

In [5]:
while org_vid.isOpened() :
    ret, frame = org_vid.read()
    
    if ret :
        sharp_frame = cv2.filter2D(src=frame, ddepth=-1, kernel=kernel)
        gray_frame = cv2.cvtColor(sharp_frame, cv2.COLOR_BGR2GRAY)
        
        detected = feature_object_detection(org_img, gray_img, frame, gray_frame, 10.5)

        cv2.imshow('Video frame', detected)

        if cv2.waitKey(int(1000/30)) & 0xFF == 27:
            break
    else :
        break
org_vid.release()
cv2.destroyAllWindows()